# BOJ 1114 - 통나무 자르기

## 문제 분석

벌목꾼 백은진이 길이 L인 통나무를 트럭에 맞게 자르려고 한다.
- K개의 위치에서만 자를 수 있음
- 최대 C번까지 자를 수 있음
- **목표: 가장 긴 조각의 길이를 최소화**

## 핵심 아이디어

1. **이분탐색**: 가능한 "가장 긴 조각의 길이"를 이분탐색
2. **그리디 전략**: 조각이 목표 길이를 초과하려 할 때 바로 이전 위치에서 자르기
3. **뒤에서부터 자르기**: 첫 번째 자르는 위치를 구할 때는 뒤에서부터 자르는 전략 사용

## ASCII 다이어그램

```
예시: L=9, 자를 수 있는 위치=[4,5], C=1

통나무: |----+----+---|
위치:   0    4    5   9

목표 길이 5로 자르기:
|----+----+---|
0    4    5   9

Case 1: 위치 4에서 자르기
[0-4] [4-9] → 길이 4, 5 (최대 5)

Case 2: 위치 5에서 자르기  
[0-5] [5-9] → 길이 5, 4 (최대 5)

뒤에서부터 자르는 전략 선택: 위치 4
```

## 알고리즘 단계

1. 자를 수 있는 위치들을 정렬
2. 이분탐색으로 가능한 최대 조각 길이 중 최솟값 찾기
3. 해당 길이로 뒤에서부터 자르는 전략으로 첫 번째 자르는 위치 구하기

### 테스트 케이스

**입력:**
```
9 2 1
4 5
```

**출력:**
```
5 4
```

### 상세 동작 과정

#### 1. 이분탐색 과정

```
초기 범위: left=1, right=9

mid=5: canCut(location=[4,5], targetLen=5, L=9, C=1)
│
├─ i=0: location[0]=4, curPos=0 → 4-0=4 ≤ 5 (OK)
├─ i=1: location[1]=5, curPos=0 → 5-0=5 ≤ 5 (OK)
└─ 마지막: L-curPos = 9-0=9 > 5 → cutCount=1 ≤ C=1 (OK)

결과: true → answer=5, right=4

left > right → 종료, answer=5
```

#### 2. 첫 번째 자르기 위치 찾기

```
findFirstCut(location=[4,5], targetLen=5, L=9, C=1)

뒤에서부터 탐색:
├─ i=1: location[1]=5, curPos=9 → 9-5=4 ≤ 5 (OK)
├─ i=0: location[0]=4, curPos=9 → 9-4=5 ≤ 5 (OK)
└─ curPos=9 > targetLen=5 → firstCut=location[0]=4

결과: 4
```


In [1]:
fun main() {
    // L, K, C 입력 받기 (통나무 길이, 자를 수 있는 위치 개수, 최대 자르기 횟수)
    val (L, K, C) = readLine()!!.split(" ").map { it.toInt() }
    
    // 자를 수 있는 위치들 입력 받기
    val location = readLine()!!.split(" ").map { it.toInt() }.toIntArray()
    location.sort() // 정렬
    
    // 이분탐색 범위: 최소 1, 최대 통나무 전체 길이
    var left = 1
    var right = L
    var answer = right
    
    // 가능한 최대 조각 길이 중 최솟값을 찾는 이분탐색
    while (left <= right) {
        val mid = left + (right - left) / 2
        
        // 길이 mid 이하로 가장 긴 조각을 만들 수 있는지 확인
        if (canCut(location, mid, L, C)) {
            answer = mid // 가능한 답 저장
            right = mid - 1 // 더 작은 길이 시도
        } else {
            left = mid + 1 // 더 큰 길이 필요
        }
    }
    
    // 첫 번째 자르는 위치 구하기
    val firstCut = findFirstCut(location, answer, L, C)
    
    println("$answer $firstCut")
}

/**
 * 주어진 targetLen으로 가장 긴 조각을 만들 수 있는지 판단
 */
fun canCut(location: IntArray, targetLen: Int, L: Int, C: Int): Boolean {
    var curPos = 0 // 현재 위치
    var cutCount = 0 // 사용한 자르기 횟수
    
    // 자를 수 있는 모든 위치를 왼쪽부터 순회
    var i = 0
    while (i < location.size) {
        // 다음 자를 수 있는 위치까지의 거리가 targetLen을 초과하면
        if (location[i] - curPos > targetLen) {
            // 첫 번째 위치까지도 targetLen을 초과하면 불가능
            if (i == 0) return false
            
            // 이전 위치에서 자르기 (가장 긴 조각이 targetLen 이하가 되도록)
            cutCount++
            if (cutCount > C) return false // 자르기 횟수 초과
            
            curPos = location[i - 1] // 자른 위치로 이동
            i-- // 현재 위치를 다시 확인 (새로운 조각 시작점에서)
        }
        i++
    }
    
    // 마지막 조각 (curPos부터 L까지)이 targetLen을 초과하는지 확인
    if (L - curPos > targetLen) {
        cutCount++ // 마지막에 한 번 더 자르기 필요
        return cutCount <= C // 총 자르기 횟수가 허용 범위 내인지
    }
    
    return true // 가장 긴 조각이 targetLen 이하로 만들 수 있음
}

/**
 * 뒤에서부터 자르는 전략으로 첫 번째 자르는 위치 찾기
 */
fun findFirstCut(location: IntArray, targetLen: Int, L: Int, C: Int): Int {
    var curPos = L // 끝에서부터 시작
    var cutCount = 0
    var firstCut = 0
    
    // 뒤에서부터 탐색
    for (i in location.size - 1 downTo 0) {
        if (curPos - location[i] > targetLen) {
            cutCount++
            firstCut = location[i + 1] // 마지막으로 자른 위치가 첫 번째 자르기
            curPos = location[i + 1]
            if (cutCount == C) break
        }
    }
    
    // 앞쪽에서 자르기가 필요한 경우
    if (curPos > targetLen && cutCount < C) {
        firstCut = location[0]
    }
    
    return firstCut
}